In [1]:
import pandas as pd
from splinter import Browser
import time
import re
import pymongo

In [2]:
# URL for scraping
url = 'https://www.espn.com/nfl/stats/player/_/view/'

#setting up browser
executable_path = {'executable_path': '../chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)

# while loop to move through webpage clicking show more, then grabbing the full page
flag=True
while flag:
    try:
        browser.click_link_by_text('Show More')
        time.sleep(2)
    except:
        print('cycle complete')
        flag= False
if flag==False:
    passing_tables= pd.read_html(browser.html)
    
# data cleaning: table comes as two, need to join
df=passing_tables[0]
df_2=passing_tables[1]


#join the cleaned tables
mf=df.join(df_2)
mf

#cleaning data: seperating team name from player names
#and putting them in a different column
mf['Name'] = mf['Name'].astype(str)
mf[['Team']]= mf['Name'].str.extract('([A-Z][A-Z]+)',expand =True)
mf['Name'] = mf.apply(lambda x: x['Name'].replace(x['Team'],""), axis=1)

#data cleaning: drop nulls for players with no QBR
mf['QBR']=mf['QBR'].fillna(0)

#add data into MongoDB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db=client.NFL_StatsDB
collection=db.passing
data1 = mf.to_dict(orient='records')
collection.insert_many(data1)

cycle complete


In [25]:
# # data cleaning: seperating team name from player name, then creating lists
# # to add the seperated names into appropriate columns
# teams_list=[]
# names_list=[]
# for item in df.Name:
#     if item[-3].isupper():
#         teams_list.append(item[-3:])
#         names_list.append(item[:-3])
#     else:
#         teams_list.append(item[-2:])
#         names_list.append(item[:-2])
# df.Name=names_list
# df['Team Name']=teams_list

##look into camelcase to help with stripping issue
##look into regex library